# Compreendendo o modelo RFM

O modelo RFM é uma técnica de segmentação de clientes que avalia três dimensões principais do comportamento do cliente: Recência , Frequência e Valor Monetário.

- **Recência (R)** : Esta métrica mede o tempo decorrido desde a última compra ou interação de um cliente com sua empresa. Quanto mais recente a compra, maior a probabilidade de repetição de negócios e engajamento do cliente.

- **Frequência (F)** : Esta métrica monitora a frequência com que um cliente faz compras ou interage com sua empresa em um determinado período. Clientes com frequências de compra mais altas costumam ser mais fiéis e valiosos.

- **Valor Monetário (M)** : Esta métrica quantifica o valor total gasto por um cliente com sua empresa. Valores monetários mais altos geralmente indicam clientes mais lucrativos que contribuem significativamente para sua receita.

In [0]:
%sql
CREATE OR REPLACE TABLE bus.gold.ota_bus_ticket_sales AS (
  SELECT
    nk_ota_localizer_id,
    fk_contact,
    date_purchase,
    time_purchase,
    cod_place_origin_departure,
    place_origin_departure,
    cod_place_destination_departure,
    CASE
      WHEN place_destination_departure IN ('Cuiabá', 'Campinas') THEN 'Rio de Janeiro'
      WHEN place_destination_departure IN ('Maceió') THEN 'Salvador'
      WHEN place_destination_departure IN ('Teresina') THEN 'Foz do Iguaçu'
      WHEN place_destination_departure IN ('Curitiba') THEN 'São Paulo'
      WHEN place_destination_departure IN ('Goiânia', 'Brasília') THEN 'Florianópolis'
      ELSE place_destination_departure
    END AS place_destination_departure,
    cod_place_origin_return,
    place_origin_return,
    cod_place_destination_return,
    place_destination_return,
    cod_departure_bus_company,
    fk_departure_ota_bus_company,
    cod_return_bus_company,
    fk_return_ota_bus_company,
    gmv_success,
    total_tickets_quantity_success
  FROM
    bus.gold.ota_bus_ticket_sales
  GROUP BY
    ALL
)

In [0]:
%sql
CREATE OR REPLACE TABLE bus.bronze.dim_clientes AS (
SELECT
  fk_contact,
  customer_code,
  customer_name,
  birth_date,
  gender,
  marital_status,
  email,
  phone_number,
  address,
  neighborhood,
  city,
  state,
  customer_group,
  income,
  CASE
    WHEN fk_contact IN (SELECT fk_contact FROM bus.bronze.dim_clientes WHERE age_group = '60+' GROUP BY ALL LIMIT 1500) THEN '26-35'
    WHEN fk_contact IN (SELECT fk_contact FROM bus.bronze.dim_clientes WHERE age_group = '46-60' GROUP BY ALL LIMIT 500) THEN '36-45'
      ELSE age_group
  END AS age_group,
  occupation,
  customer_status,
  last_update_date,
  active_record_indicator
FROM
  bus.bronze.dim_clientes
)

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW customers_cte AS (
SELECT
  fk_contact,
  COUNT(DISTINCT nk_ota_localizer_id) AS orders,
  ROUND(SUM(gmv_success), 2) AS order_value,
  MAX(date_purchase) AS last_order_date
FROM
  bus.gold.ota_bus_ticket_sales
GROUP BY
  ALL
  )

In [0]:
%py
rfm_df = spark.sql("""
WITH rfm_cte AS (
  SELECT
    *,
    PERCENT_RANK() OVER (Order by last_order_date) AS recency,
    PERCENT_RANK() OVER (Order by orders) AS frequency,
    PERCENT_RANK() OVER (Order by order_value) AS monetary
  FROM
    customers_cte
),
score_cte AS (
  SELECT
    *,
    (0.2 * recency) + (0.4 * frequency) + (0.4 * monetary) AS rfm_wights,
    PERCENT_RANK() OVER (
        ORDER BY (0.2 * recency) + (0.4 * frequency) + (0.4 * monetary)
      ) AS norm_rfm_score
  FROM
    rfm_cte
)
SELECT
  fk_contact,
  orders,
  COALESCE(order_value, 0) AS order_value,
  CAST(last_order_date AS DATE) AS last_order_date,
  recency AS Recencia,
  CASE
    WHEN recency = 0 THEN 'Sem Recência'
    WHEN recency <= 0.50 THEN 'Recência baixa'
    WHEN recency <= 0.85 THEN 'Recência média'
    WHEN recency <= 1 THEN 'Alta Recência'
    ELSE 'Sem Recência'
  END AS classificacao_recencia,
  frequency AS Frequencia,
  CASE
    WHEN frequency = 0 THEN 'Sem Frequência'
    WHEN frequency <= 0.50 THEN 'Frequência baixa'
    WHEN frequency <= 0.85 THEN 'Frequência média'
    WHEN frequency <= 1 THEN 'Alta Frequência'
    ELSE 'Sem Frequência'
  END AS classificacao_Frequencia,
  monetary AS Monetario,
  CASE
    WHEN monetary = 0 THEN 'Sem valor'
    WHEN monetary <= 0.50 THEN 'Baixo Valor'
    WHEN monetary <= 0.85 THEN 'Médio Valor'
    WHEN monetary <= 1 THEN 'Valor alto'
    ELSE 'Sem valor'
  END AS classificacao_Monetario,
  norm_rfm_score,
  CASE
    WHEN orders = 0 THEN 'E'
    WHEN norm_rfm_score >= 0.95 THEN 'A'
    WHEN norm_rfm_score >= 0.80 THEN 'B'
    WHEN norm_rfm_score >= 0.50 THEN 'C'
    ELSE 'D'
  END AS Segmento,
  CASE
    WHEN recency = 0 THEN 'Clientes Perdidos'
    WHEN recency > 0.85 AND frequency > 0.85 AND monetary > 0.85 THEN 'Clientes Estratégicos'
    WHEN recency <= 0.20 AND monetary <= 0.20 THEN 'Clientes Inativos'
    WHEN frequency <= 0.50 AND monetary <= 0.50 THEN 'Clientes Ocasional'
    WHEN monetary > 0.85 AND recency <= 0.50 THEN 'Clientes em Declínio'
    WHEN monetary > 0.85 AND recency <= 0.50 THEN 'Clientes Premium em Risco'
    WHEN frequency > 0.50 AND monetary > 0.50 THEN 'Clientes Regulares'
    WHEN recency > 0.85 AND monetary <= 0.50 THEN 'Novos Clientes'
    WHEN recency > 0.85 AND monetary > 0.50 THEN 'Clientes em Potencial'
    WHEN frequency > 0.85 AND monetary > 0.50 THEN 'Clientes Leais'
    ELSE 'Clientes Ativos'
  END AS segmento_comercial
FROM
  score_cte
ORDER BY
  norm_rfm_score DESC
""")

In [0]:
%python
rfm_pdf = rfm_df.toPandas()
import seaborn as sns
import matplotlib.pyplot as plt

sns.pairplot(rfm_pdf[["Recencia", "Frequencia", "Monetario", "segmento_comercial"]],
             hue="segmento_comercial",
             palette="Set2",
             plot_kws={'alpha':0.7, 's':60})

plt.show()

Mapeamento e Classificação dos Clientes (10 grupos RFM → nomes):
| Segmento RFM                          | Nome Profissional             |
| ------------------------------------- | ----------------------------- |
| Alta Recência, Alta Freq., Alto Valor | **Clientes Estratégicos**     |
| Alta Freq., Médio Valor               | **Clientes Leais**            |
| Alto Valor, mas Recência baixa        | **Clientes Premium em Risco** |
| Recente, Baixo Valor                  | **Novos Clientes**            |
| Recente, Médio Valor                  | **Clientes em Potencial**     |
| Frequência média, Valor médio         | **Clientes Regulares**        |
| Valor alto, Recência baixa            | **Clientes em Declínio**      |
| Baixa Freq., Baixo Valor              | **Clientes Ocasional**        |
| Longa recência, pouco gasto           | **Clientes Inativos**         |
| Muito tempo sem atividade             | **Clientes Perdidos**         |


In [0]:
%python

# display(
#     rfm_df.filter(
#         rfm_df["segmento_comercial"].isin(["Clientes Inativos"])
#     )
# )

rfm_df.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("bus.gold.rmf_customer_segmentation")

In [0]:
%sql
SELECT * FROM bus.gold.rmf_customer_segmentation